In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))

from data_loader import get_db_engine, load_gold_data

engine = get_db_engine()

try:
    gold_data = load_gold_data(engine)
    df_customers = gold_data['customers']
    df_products = gold_data['products']
    df_sales = gold_data['sales']

    print("\nPrimeiras 5 linhas do DataFrame de Vendas:")
    print(df_sales.head())
    print("\nPrimeiras 5 linhas do DataFrame de Clientes:")
    print(df_customers.head())
    print("\nPrimeiras 5 linhas do DataFrame de Produtos:")
    print(df_products.head())

except Exception as e:
    print(f"Não foi possível carregar os dados: {e}")
finally:
    engine.dispose()
    print("Conexão com o banco de dados encerrada no notebook.")

Carregando dados da camada Gold...
Dados da camada Gold carregados com sucesso!
Shape df_customers: (18484, 10)
Shape df_products: (295, 11)
Shape df_sales: (60398, 9)

Primeiras 5 linhas do DataFrame de Vendas:
  order_number  product_key  customer_key  order_date shipping_date  \
0      SO43697           20         10769  2010-12-29    2011-01-05   
1      SO43698            9         17390  2010-12-29    2011-01-05   
2      SO43699            9         14864  2010-12-29    2011-01-05   
3      SO43700           41          3502  2010-12-29    2011-01-05   
4      SO43701            9             4  2010-12-29    2011-01-05   

     due_date  sales_amount  quantity  sls_price  
0  2011-01-10          3578         1       3578  
1  2011-01-10          3400         1       3400  
2  2011-01-10          3400         1       3400  
3  2011-01-10           699         1        699  
4  2011-01-10          3400         1       3400  

Primeiras 5 linhas do DataFrame de Clientes:
   custom

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

sales_with_products = pd.merge(df_sales, df_products, on='product_key', how='left')

print("\nHead de sales_with_products:")
print(sales_with_products.head())
print(f"Shape: {sales_with_products.shape}")

full_sales_data = pd.merge(sales_with_products, df_customers, on='customer_key', how='left')

print("\nHead de full_sales_data:")
print(full_sales_data.head())
print(f"Shape: {full_sales_data.shape}")


sales_time_series = full_sales_data.groupby([pd.Grouper(key='order_date', freq='ME'), 'product_line'])['sales_amount'].sum().reset_index()

fig_line_time_series = px.line(
    sales_time_series,
    x='order_date',
    y='sales_amount',
    color='product_line',
    title='Vendas Totais ao Longo do Tempo por Linha de Produto',
    labels={'order_date': 'Data do Pedido', 'sales_amount': 'Total de Vendas', 'product_line': 'Linha de Produto'}
)
fig_line_time_series.update_layout(xaxis_title="Data", yaxis_title="Total de Vendas")
fig_line_time_series.show()



Head de sales_with_products:
  order_number  product_key  customer_key order_date shipping_date   due_date  \
0      SO43697           20         10769 2010-12-29    2011-01-05 2011-01-10   
1      SO43698            9         17390 2010-12-29    2011-01-05 2011-01-10   
2      SO43699            9         14864 2010-12-29    2011-01-05 2011-01-10   
3      SO43700           41          3502 2010-12-29    2011-01-05 2011-01-10   
4      SO43701            9             4 2010-12-29    2011-01-05 2011-01-10   

   sales_amount  quantity  sls_price  product_id product_number  \
0          3578         1       3578         310     BK-R93R-62   
1          3400         1       3400         346     BK-M82S-44   
2          3400         1       3400         346     BK-M82S-44   
3           699         1        699         337     BK-R50B-62   
4          3400         1       3400         346     BK-M82S-44   

              product_name category_id category     subcategory maintenance  \
0

In [ ]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

df = full_sales_data.copy()
df['order_date'] = pd.to_datetime(df['order_date']) 

app = dash.Dash(__name__)
app.title = "BI Interativo de Vendas"

app.layout = html.Div([
    html.H2("💼 Dashboard Interativo de Vendas", style={'textAlign': 'center'}),

    html.Div([
        html.Div([
            html.Label("Selecionar Gênero:"),
            dcc.Dropdown(
                id='gender_filter',
                options=[{'label': g, 'value': g} for g in sorted(df['gender'].unique())] + [{'label': 'Todos', 'value': 'All'}],
                value='All',
                clearable=False
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            html.Label("Selecionar País:"),
            dcc.Dropdown(
                id='country_filter',
                options=[{'label': c, 'value': c} for c in sorted(df['country'].unique())] + [{'label': 'Todos', 'value': 'All'}],
                value='All',
                clearable=False
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
    ], style={'padding': '20px'}),

    html.Div([
        dcc.Graph(id='bar_chart'),
    ], style={'width': '100%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            dcc.Graph(id='pie_chart'),
        ], style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(id='timeline_chart'),
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),
    ])
])

@app.callback(
    Output('bar_chart', 'figure'),
    Output('pie_chart', 'figure'),
    Output('timeline_chart', 'figure'),
    Input('gender_filter', 'value'),
    Input('country_filter', 'value')
)
def update_charts(selected_gender, selected_country):
    
    filtered = df.copy()
    if selected_gender != 'All':
        filtered = filtered[filtered['gender'] == selected_gender]
    if selected_country != 'All':
        filtered = filtered[filtered['country'] == selected_country]

    bar_data = filtered.groupby(['country', 'category'])['sales_amount'].sum().reset_index()
    bar_fig = px.bar(
        bar_data,
        x='country',
        y='sales_amount',
        color='category',
        barmode='group',
        title='Total de Vendas por Categoria e País'
    )

    pie_data = filtered.groupby('gender')['sales_amount'].sum().reset_index()
    pie_fig = px.pie(
        pie_data,
        values='sales_amount',
        names='gender',
        title='Distribuição de Vendas por Gênero',
        hole=0.3
    )

    time_data = filtered.groupby(filtered['order_date'].dt.to_period('M'))['sales_amount'].sum().reset_index()
    time_data['order_date'] = time_data['order_date'].astype(str)
    time_fig = px.line(
        time_data,
        x='order_date',
        y='sales_amount',
        title='Evolução das Vendas ao Longo do Tempo',
        markers=True
    )

    return bar_fig, pie_fig, time_fig

if __name__ == '__main__':
    app.run(debug=True)


In [17]:
# --- 1. Combinar df_sales com df_products para obter o custo do produto e o nome ---
# Vamos usar 'product_key' para fazer a junção
# Selecionamos apenas as colunas necessárias de df_products para a junção e para o cálculo do lucro
sales_with_product_info = pd.merge(
    df_sales,
    df_products[['product_key', 'product_number', 'product_name', 'product_cost', 'category', 'subcategory']],
    on='product_key',
    how='left'
)

print("\nHead de 'sales_with_product_info' (após junção):")
print(sales_with_product_info.head())
print(f"Shape: {sales_with_product_info.shape}")

# Preencha quaisquer NaNs que podem ter surgido do merge (se um product_key não foi encontrado)
# Você pode decidir como tratar isso (0, média, mediana, etc.). Para lucro, 0 é seguro para custo/preço.
sales_with_product_info.fillna({'product_cost':0}, inplace=True)
sales_with_product_info.fillna({'sls_price':0}, inplace=True)
sales_with_product_info.fillna({'quantity':0}, inplace=True)


print(sales_with_product_info[['product_number', 'product_name', 'product_cost', 'sls_price', 'quantity']].head())
# Calcular o lucro unitário (preço de venda - custo)
sales_with_product_info['unit_profit'] = sales_with_product_info['sls_price'] - sales_with_product_info['product_cost']

# Calcular o lucro total por linha de venda (lucro unitário * quantidade)
sales_with_product_info['total_profit'] = sales_with_product_info['unit_profit'] * sales_with_product_info['quantity']

print("\nHead de 'sales_with_product_info' (após cálculo do lucro):")
print(sales_with_product_info[['order_number', 'product_number', 'product_name', 'sls_price', 'product_cost', 'quantity', 'unit_profit', 'total_profit']].head())


# --- 3. Agrupar o lucro por product_number e product_name ---
# Vamos somar o 'total_profit' para cada 'product_number'
profit_by_product = sales_with_product_info.groupby(['product_number', 'product_name'])['total_profit'].sum().reset_index()

# Ordenar os produtos pelo lucro, do maior para o menor
profit_by_product = profit_by_product.sort_values(by='total_profit', ascending=False)

print("\nLucro Total por Produto (Top 10):")
print(profit_by_product.head(10))

# --- 4. Plotar o Lucro por Product_Number usando Plotly ---

# Para um grande número de produtos, um gráfico de barras horizontal é mais legível.
# Se tiver muitos produtos, considere plotar apenas o TOP N.
top_n = 20 # Número de produtos a exibir
profit_to_plot = profit_by_product.head(top_n) if len(profit_by_product) > top_n else profit_by_product

fig = px.bar(
    profit_to_plot,
    x='total_profit',
    y='product_name', # Usar o nome do produto no eixo Y para melhor legibilidade
    orientation='h', # Barras horizontais
    title=f'Lucro Total por Produto (Top {len(profit_to_plot)})',
    labels={'total_profit': 'Lucro Total', 'product_name': 'Nome do Produto'},
    hover_data={'product_number': True, 'total_profit': ':.2f'} # Mostra o product_number e lucro formatado no hover
)

# Inverter o eixo Y para que o produto com maior lucro fique no topo
fig.update_layout(yaxis={'categoryorder':'total ascending'})

fig.show()

# --- Análise Adicional: Lucro por Categoria de Produto ---
# Também é interessante ver qual categoria de produto gera mais lucro
profit_by_category = sales_with_product_info.groupby('category')['total_profit'].sum().reset_index()
profit_by_category = profit_by_category.sort_values(by='total_profit', ascending=False)

print("\nLucro Total por Categoria de Produto:")
print(profit_by_category)

fig_category_profit = px.bar(
    profit_by_category,
    x='category',
    y='total_profit',
    title='Lucro Total por Categoria de Produto',
    labels={'total_profit': 'Lucro Total', 'category': 'Categoria do Produto'},
    color='category' # Cores diferentes para cada categoria
)
fig_category_profit.show()





Head de 'sales_with_product_info' (após junção):
  order_number  product_key  customer_key order_date shipping_date   due_date  \
0      SO43697           20         10769 2010-12-29    2011-01-05 2011-01-10   
1      SO43698            9         17390 2010-12-29    2011-01-05 2011-01-10   
2      SO43699            9         14864 2010-12-29    2011-01-05 2011-01-10   
3      SO43700           41          3502 2010-12-29    2011-01-05 2011-01-10   
4      SO43701            9             4 2010-12-29    2011-01-05 2011-01-10   

   sales_amount  quantity  sls_price product_number             product_name  \
0          3578         1       3578     BK-R93R-62         Road-150 Red- 62   
1          3400         1       3400     BK-M82S-44  Mountain-100 Silver- 44   
2          3400         1       3400     BK-M82S-44  Mountain-100 Silver- 44   
3           699         1        699     BK-R50B-62       Road-650 Black- 62   
4          3400         1       3400     BK-M82S-44  Mountain-1


Lucro Total por Categoria de Produto:
      category  total_profit
1        Bikes      11109565
0  Accessories        439510
2     Clothing        136682


In [21]:
!streamlit run app.py


^C
